In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import os
import glob
import re
import pymongo
import yaml
import src.helper as helper


In [66]:
config = yaml.load(open("config.yaml", "r").read())
textbook_src = config["TextbookSource"]["path"]
config

{'TextbookSource': {'archive': 'CoeusTextSrc_20170906.zip',
  'availability': ['Google Drive'],
  'path': 'E:\\Study\\14_LDA\\20_Materials\\CoeusTextSrc'}}

# Build text data in to database

In [83]:
from pymongo import MongoClient
client = MongoClient()
if "textbooks" in client.database_names():
    client.drop_database("textbooks")
db = client.textbooks
corpus = db.corpus

In [84]:
fname_pat = re.compile("(?P<version>.*?)_(?P<subject>.*?)_G\d_(?P<semester>.*?)_.*")
txt_files = glob.glob(textbook_src + "/*.txt")
dataObj = []
for txt_path in txt_files:
    basename = os.path.basename(txt_path)
    fname_mat = fname_pat.match(basename)    
    if not fname_mat:
        print(txt_path)
    textObj = helper.parse_text_file(txt_path)
    textObj.update(fname_mat.groupdict())
    dataObj.append(textObj)
ret = corpus.insert_many(dataObj)    
print("insert %d text into database" % len(ret.inserted_ids))

insert 1307 text into database


In [79]:
corpus.find_one({})

{'_id': ObjectId('59af956c1175b70be073e5b0'),
 'numgrade': 1,
 'numlesson': 1,
 'semester': '0982',
 'strtitle': '一\u3000春雨',
 'subject': 'Chinese',
 'text': '下雨了！下雨了！\n種子說：「下吧！下吧！我們要發芽。」小草說：「下吧！下吧！我們要長大。」\n桃樹說：「下吧！下吧！我們要開花。」\n下雨了！下雨了！大家都說：「春天的小雨，真好！」',
 'version': 'HanLin'}

In [82]:
list(corpus.find({"subject": {"$eq": "Chinese"}}, {"strtitle": 1}).sort([("numgrade", 1), ("numlesson", 1)]).limit(10))

[{'_id': ObjectId('59af956c1175b70be073e5b0'), 'strtitle': '一\u3000春雨'},
 {'_id': ObjectId('59af956c1175b70be073e5be'), 'strtitle': '一、木頭人'},
 {'_id': ObjectId('59af956c1175b70be073e74f'), 'strtitle': '一 小樹 黃長安'},
 {'_id': ObjectId('59af956c1175b70be073e75d'), 'strtitle': '拍拍手'},
 {'_id': ObjectId('59af956c1175b70be073e8e0'), 'strtitle': '小熊過冬'},
 {'_id': ObjectId('59af956c1175b70be073e8ec'), 'strtitle': '一 手拉手'},
 {'_id': ObjectId('59af956c1175b70be073e5b6'), 'strtitle': '二\u3000找春天'},
 {'_id': ObjectId('59af956c1175b70be073e5bf'), 'strtitle': '雲和花'},
 {'_id': ObjectId('59af956c1175b70be073e755'), 'strtitle': '二 我要長大'},
 {'_id': ObjectId('59af956c1175b70be073e75e'), 'strtitle': '吹泡泡'}]